In [4]:
import requests
from bs4 import BeautifulSoup
from IPython.display import display, HTML, clear_output
import random
import time
import pandas as pd
import json
import ipywidgets as widgets
from jupyter_ui_poll import ui_events

event_info = {
    'type': '',
    'description': '',
    'time': -1
}

#Source：Moodle：Button demo code.ipynb
def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

#Source：Moodle：Button demo code.ipynb
def register_event(btn):
    # display button description in output area
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

def generate_problem(question_count):
    if question_count <= 8:
        num1 = random.randint(0, 10)
        num2 = random.randint(0, 10)
        num3 = random.randint(0, 10)
    elif 9 <= question_count <= 15:
        num1 = random.randint(11, 50)
        num2 = random.randint(11, 50)
        num3 = random.randint(0, 10)
    else:
        # Handle the case when question_count is outside the specified ranges
        raise ValueError("Invalid question_count")
    operator1 = random.choice(['+', '-'])
    operator2 = random.choice(['+', '-','*'])
    return num1, num2, num3, operator1, operator2

# Function to send data to a Google Form
def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

# Function for retrieving user details if they consent
def get_details():
    #with details_output:
    display(HTML("What is your name?"))
    name = input('Name:')
    
    display(HTML("What is your age?"))
    age = input('Age:')
    
    display(HTML("What is your gender?"))
    gender = input('Gender:')
    
    return name, age, gender

# Display number and operator to give a question
def display_problem(num1, operator1, num2, operator2, num3):
    # Display the problem using HTML-like formatting
    display(HTML(f"<span style='color: red; font-size: 30px;'>{num1}</span>"))
    time.sleep(2)
    clear_output(wait=True)

    display(HTML(f"<span style='color: red; font-size: 30px;'>{operator1}{num2}</span>"))
    time.sleep(2)
    clear_output(wait=True)
    
    display(HTML(f"<span style='color: red; font-size: 30px;'>{operator2}{num3}</span>"))
    time.sleep(1)
    clear_output(wait=True)

#use to confirm user input is int or float
def type_check(int):
    type_int = type(int)
    print(type_int)
    if type_int == type(1):
        return int
    elif type_int == type(1.0):
        return int
    else:
        return -100000 #must be wrong

#Colab by group member Qianzhi Sang

# code for consent botton 

def willingness_check(btn1,btn2):
    # we need to set up each button
    # to call the register_event
    # function when clicked
    btn1.on_click(register_event) 
    btn2.on_click(register_event) 

    myhtml1 = HTML("<span style='font-size: 20px;'> Would you like to enter your details to be used for research purposes? </span>")
    display(myhtml1)


    panel = widgets.HBox([btn1, btn2])
    display(panel)

    result = wait_for_event(timeout=60)
    clear_output()

    if result['description']!="":
        print(f"User clicked: {result['description']}")
        return(result['description'])
    else:
        print("User did not click in time")
    

#Source:Moodle Week 6, Using_buttons_and_text_input_fix.ipynb
def text_input(prompt=None):
    text_input = widgets.Text(description=prompt, style= {'description_width': 'initial'})
    import warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    text_input.on_submit(register_text_input_event)
    display(text_input)
    event = wait_for_event(timeout=10)
    text_input.disabled = True
    return event['description']

#Source:Moodle Week 6, Using_buttons_and_text_input_fix.ipynb
def register_text_input_event(text_input):
    event_info['type'] = "text_entry"
    event_info['description'] = text_input.value
    event_info['time'] = time.time()
    return

# Main calculation test
def main():
    correct_answers = 0
    score = 0
    total_time = 0
    num_problems = 15  # You can change the number of problems

    display(HTML(f"<span style='color: black; font-size: 20px;'>Math Calculation Ability Test: Here is the example for your further step calculation, if you see '2+3*2' it would be (2+3)*2</span>"))

    name,age,gender = get_details()

    #The two buttons
    btn1 = widgets.Button(description="Yes")
    btn2 = widgets.Button(description="No")

    if_consent = willingness_check(btn1,btn2)
    if if_consent == 'Yes':
        #Time list that store the time spend each time
        time_list_set1 = []
        time_list_set2 = []

        for i in range(1, num_problems + 1):
            num1, num2, num3, operator1, operator2 = generate_problem(i)

            display(HTML(f"<span style='color: black; font-size: 20px;'>Get ready for problem {i}...</span>"))
            time.sleep(2)
            clear_output(wait=True)

            # Display the problem for 2 seconds
            display_problem(num1, operator1, num2, operator2, num3 )

            # Record start time
            start_time = time.time()
        
            # Ask the user to solve the problem
            display(HTML(f"<span style='color: black; font-size: 20px;'>What's the answer? </span>"))
            

            #Use the Text_input_function
            user_input = text_input(prompt='The answer is:')
            user_answer = type_check(int(user_input)) 
            #now user input must be int/float to prevent bug, if not,the answer is a 100% wrong answer to punish
        
            # Record end time
            end_time = time.time()
            # Calculate the time spent on the current question
            time_spent = end_time - start_time
        
            if 1 <= i <= 8:
                time_list_set1.append(time_spent)
            elif 9 <= i <= 15:
                time_list_set2.append(time_spent)
        
            total_time += time_spent
        
            # Calculate the correct answer
            if operator1 == '+' and operator2 == '+':
                correct_answer = num1 + num2 + num3
            elif operator1 == '+' and operator2 == '-':
                correct_answer = num1 + num2 - num3
            elif operator1 == '+' and operator2 == '*':
                correct_answer = (num1 + num2 )* num3
            elif operator1 == '-' and operator2 == '+':
                correct_answer = num1 - num2 + num3
            elif operator1 == '-' and operator2 == '-':
                correct_answer = num1 - num2 - num3
            elif operator1 == '-' and operator2 == '*':
                correct_answer = (num1 - num2) * num3

            # Check if the user's answer is correct
            if user_answer == correct_answer:
                print(f"Correct! You earned 1 point. Time: {time_spent:.2f} seconds\n")
                correct_answers += 1
                score += 1
            else:
                print(f"Incorrect. The correct answer is {correct_answer}. No point earned. Time: {time_spent:.2f} seconds\n")

    # Calculate the average time spent
    average_time_set1 = sum(time_list_set1) / len(time_list_set1) if time_list_set1 else 0
    average_time_set2 = sum(time_list_set2) / len(time_list_set2) if time_list_set2 else 0
        
    # Display the results using HTML
    display(HTML(f"<p style='color: blue; font-size: 20px;'>You got {correct_answers} out of {num_problems} problems correct.</p>"))
    display(HTML(f"<p style='color: blue; font-size: 20px;'>Your total score is {score}.</p>"))
    display(HTML(f"<p style='color: blue; font-size: 20px;'>Total time spent on calculation: {total_time:.2f} seconds.</p>"))
    display(HTML(f"<p style='color: blue; font-size: 20px;'>Average time spent for questions 1-8: {average_time_set1:.2f} seconds.</p>"))
    display(HTML(f"<p style='color: blue; font-size: 20px;'>Average time spent for questions 9-15: {average_time_set2:.2f} seconds.</p>"))

    myhtml1 = HTML("<span style='font-size: 20px;'> Would you like to enter your details to be used for research purposes? </span>")
    display(myhtml1)

    panel = widgets.HBox([btn1, btn2])
    display(panel)

    result = wait_for_event(timeout=60)
    clear_output()

    #indentation 
    if result['description']!="":
        print(f"User clicked: {result['description']}")
    else:
        print("User did not click in time")


    # Dictionaries for user information and test results
    data_dict1 = {'Name': name, 'Age': age, 'Gender': gender}
    data_dict2 = {'Average_time_easy': average_time_set1, 'Average_time_hard': average_time_set2, 'Total_time': total_time, 'Final_grade': score}

    # Create a new dictionary by copying data_dict1 and updating it with data_dict2
    new_data_dict = data_dict1.copy()
    new_data_dict.update(data_dict2)

    # Send data to Google Form
    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSdtOV3JVnW1jtoSxjP8HsBm9lJUupjaJ3ck3Fem95XEkRm28g/viewform?usp=sf_link'
    if_success = send_to_google_form(new_data_dict, form_url)
    return if_success

main()

User clicked: Yes


True